In [ ]:
from flask import Flask, request, jsonify
import pyttsx3
import speech_recognition as sr
import os
import datetime
import sqlite3
import smtplib
import shutil
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib.units import inch

app = Flask(_name_)

# Initialize the text-to-speech engine
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

def speak(audio):
    engine.say(audio)
    engine.runAndWait()

def wishMe():
    hour = int(datetime.datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning!")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon!")
    else:
        speak("Good Evening!")
    speak("hello valliappan")
    speak("I am Friday, your personal assistant.")

def takeCommand():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source)

    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language='en-in')
        print(f"User said: {query}\n")

    except Exception as e:
        print("Say that again please...")
        return "None"
    return query

def create_database():
    conn = sqlite3.connect('your_database.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS database (
                        id INTEGER PRIMARY KEY,
                        class_id TEXT,
                        time TEXT
                    )''')
    conn.commit()
    conn.close()

# Function to insert data into the database
def insert_data(class_id, time):
    conn = sqlite3.connect('your_database.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO database (class_id, time) VALUES (?, ?)", (class_id, time))
    conn.commit()
    conn.close()

def fetch_data_from_database():
    # Connect to the SQLite database
    conn = sqlite3.connect('your_database.db')
    cursor = conn.cursor()

    # Fetch data from the database
    cursor.execute("SELECT * FROM database")
    data = cursor.fetchall()

    # Close the database connection
    conn.close()

    return data

def create_pdf(data):
    # Create a PDF document
    pdf_filename = "data_report.pdf"
    doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
    elements = []

    # Define the table structure
    table_data = [['S No', 'Class ID', 'Time']]
    for row in data:
        table_data.append(list(row))

    # Create the table
    table = Table(table_data, colWidths=[1 * inch, 1 * inch, 2 * inch])

    # Add style to the table
    style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                        ('GRID', (0, 0), (-1, -1), 1, colors.black)])

    table.setStyle(style)
    elements.append(table)

    # Build the PDF document
    doc.build(elements)

    print(f"PDF report generated: {pdf_filename}")

def send_emergency_message(message):
    sender_email = "vvalliappan2004@gmail.com"
    sender_password = "qksg sulw izbh jzzv"
    receiver_email = "manodharshan.k@gmail.com"

    subject = "Emergency Alert!"
    body = message

    message1 = f"Subject: {subject}\n\n{body}"

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, receiver_email, message1)
    server.quit()

@app.route('/assistant', methods=['POST'])
def assistant():
    data = request.json
    query = data['query'].lower()

    if 'open yolo' in query:
        # Initiating object detection protocol
        speak("Initiating object detection protocol")
        return jsonify({"message": "Object detection initiated."})

    elif 'open youtube' in query:
        webbrowser.open("https://www.youtube.com/")
        speak('Opening YouTube')
        return jsonify({"message": "YouTube opened."})

    elif 'open google' in query:
        speak('Opening Google')
        webbrowser.open("https://www.google.com/")
        return jsonify({"message": "Google opened."})

    elif 'play music' in query:
        music_dir = 'C:\\Music'  # Change this path to your music directory
        songs = os.listdir(music_dir)
        os.startfile(os.path.join(music_dir, songs[0]))
        return jsonify({"message": "Playing music."})

    elif 'tell me the time now' in query:
        strTime = datetime.datetime.now().strftime("%H:%M:%S")
        speak(f"The time is {strTime}")
        return jsonify({"message": f"The time is {strTime}."})

    elif 'emergency' in query:
        send_emergency_message("Emergency! Please help me.")
        print("Emergency message sent.")
        return jsonify({"message": "Emergency message sent."})

    elif 'switch off' in query:
        speak("Goodbye!")
        # Fetch data from the database
        data = fetch_data_from_database()
        # Create a PDF document with the fetched data
        create_pdf(data)
        return jsonify({"message": "Assistant turned off and logs exported to PDF."})

    return jsonify({"message": "Query processed."})

if _name_ == '_main_':
    create_database()
    wishMe()
    app.run(host='0.0.0.0',port=5000)

In [2]:
from flask import Flask, request
from flask_cors import CORS
import cv2
import numpy as np

app = Flask(__name__)
CORS(app)

@app.route('/receive_frame', methods=['POST'])
def receive_frame():
    frame_data = request.json['frame']
    frame_bytes = np.frombuffer(frame_data, dtype=np.uint8)
    frame = cv2.imdecode(frame_bytes, cv2.IMREAD_COLOR)
    
    # Process the received frame (e.g., display it)
    cv2.imshow('Received Frame', frame)
    cv2.waitKey(1)
    
    return "Frame received by server"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)  # Listen on all network interfaces

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\al\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import cv2
import requests
import numpy as np

# RTSP URL of the video stream (replace with your RTSP URL)
rtsp_url = 'rtsp://your_camera_ip_address/live/stream'

cap = cv2.VideoCapture(rtsp_url)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to bytes
    frame_bytes = cv2.imencode('.jpg', frame)[1].tobytes()
    
    # Send frame to server
    server_url = 'http://your_laptop_ip_address:5000/receive_frame'  # Change this to your laptop's IP address
    response = requests.post(server_url, json={"frame": frame_bytes})
    print("Response:", response.text)

cap.release()
cv2.destroyAllWindows()

In [ ]:
import picamera
import io

def load_video():
    # Open the Raspberry Pi camera module
    with picamera.PiCamera() as camera:
        camera.resolution = (640, 480)
        camera.framerate = 24
        camera.start_preview()

        # Start streaming the video over RTSP
        stream = io.BytesIO()
        camera.start_recording(stream, format='h264')
        yield b'RTSP/1.0 200 OK\r\n' + b'Content-Type: video/h264\r\nContent-Length: 0\r\n\r\n'
        while True:
            # Write the video frames to the stream
            data = stream.read(1024)
            if not data:
                break
            yield data
        camera.stop_recording()